In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from datetime import datetime

# Function to get the list of S&P 500 companies
def get_sp500_companies():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    tables = pd.read_html(response.text)
    df = tables[0]
    return df['Symbol'].tolist()



ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Function to perform DCF analysis on a single stock
def dcf_analysis(ticker):
    try:
        # Fetch stock data
        stock = yf.Ticker(ticker)

        # Get historical Free Cash Flow (FCF)
        cashflow = stock.cashflow
        if cashflow.empty:
            return None

        # Calculate Free Cash Flow (Operating Cash Flow - Capital Expenditures)
        operating_cf = cashflow.loc['Total Cash From Operating Activities']
        capex = cashflow.loc['Capital Expenditures']
        fcf = operating_cf + capex  # CapEx is usually negative in cash flow statements
        fcf = fcf.dropna()

        if fcf.empty or len(fcf) < 2:
            return None

        # Convert FCF to list and calculate growth rates
        fcf_values = fcf.values.astype(float)
        growth_rates = []
        for i in range(len(fcf_values)-1):
            if fcf_values[i+1] == 0:
                continue
            growth = (fcf_values[i] - fcf_values[i+1]) / abs(fcf_values[i+1])
            growth_rates.append(growth)

        if not growth_rates:
            avg_growth_rate = 0.02  # Default to 2% if unable to calculate
        else:
            avg_growth_rate = np.mean(growth_rates)
            if np.isnan(avg_growth_rate) or avg_growth_rate < -0.5 or avg_growth_rate > 0.5:
                avg_growth_rate = 0.02  # Default to 2% for extreme values

        # Project future FCFs for the next 5 years
        last_fcf = fcf_values[0]
        projected_fcfs = []
        for i in range(1, 6):
            projected_fcf = last_fcf * ((1 + avg_growth_rate) ** i)
            projected_fcfs.append(projected_fcf)

        # Calculate WACC (simplified as cost of equity using CAPM)
        beta = stock.info.get('beta', 1)
        risk_free_rate = 0.03  # 3% as the risk-free rate (10-year Treasury yield)
        market_return = 0.08  # Estimated market return
        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

        # Validate cost of equity
        if cost_of_equity <= 0 or cost_of_equity > 0.15:
            cost_of_equity = 0.08  # Default to 8% if invalid

        discount_rate = cost_of_equity

        # Calculate Terminal Value using Gordon Growth Model
        terminal_growth_rate = 0.02  # 2% perpetual growth rate
        terminal_value = projected_fcfs[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)

        # Discount future FCFs to present value
        pv_fcfs = [fcf / ((1 + discount_rate) ** (i+1)) for i, fcf in enumerate(projected_fcfs)]
        pv_terminal_value = terminal_value / ((1 + discount_rate) ** 5)

        # Calculate Enterprise Value
        enterprise_value = sum(pv_fcfs) + pv_terminal_value

        # Get net debt from balance sheet
        balance_sheet = stock.balance_sheet
        if balance_sheet.empty:
            return None

        total_debt = balance_sheet.loc['Short Long Term Debt'].iloc[0] + balance_sheet.loc['Long Term Debt'].iloc[0]
        cash_and_equiv = balance_sheet.loc['Cash'].iloc[0]
        net_debt = total_debt - cash_and_equiv

        # Calculate Equity Value
        equity_value = enterprise_value - net_debt

        # Get number of shares outstanding
        shares_outstanding = stock.info.get('sharesOutstanding', 0)
        if shares_outstanding == 0:
            return None

        intrinsic_value_per_share = equity_value / shares_outstanding

        # Get current stock price
        current_price = stock.info.get('currentPrice', None)
        if current_price is None:
            return None

        # Determine if the stock is undervalued
        undervalued = intrinsic_value_per_share > current_price

        return {
            'Ticker': ticker,
            'Intrinsic Value': intrinsic_value_per_share,
            'Current Price': current_price,
            'Undervalued': undervalued
        }

    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        return None

# Main script
if __name__ == "__main__":
    sp500_companies = get_sp500_companies()
    results = []

    for ticker in sp500_companies:
        print(f"Processing {ticker}...")
        analysis = dcf_analysis(ticker)
        if analysis:
            results.append(analysis)

    # Convert results to DataFrame
    df_results = pd.DataFrame(results)
    df_undervalued = df_results[df_results['Undervalued'] == True]

    # Save results to CSV
    df_results.to_csv('sp500_dcf_analysis.csv', index=False)
    df_undervalued.to_csv('sp500_undervalued_stocks.csv', index=False)

    print("Analysis complete.")
    print(f"Total stocks analyzed: {len(results)}")
    print(f"Potentially undervalued stocks: {len(df_undervalued)}")
    print("Results saved to 'sp500_dcf_analysis.csv' and 'sp500_undervalued_stocks.csv'.")
